<strong>주가 예측 프로젝트 prac 2</strong>

- 딕셔너리 사용 <br>
<strong> 수행시간 O(1)</strong> 
- 과제 1: 0초
- 과제 2: 3분 9초
- 과제 3: 9초

-> 과제2의 경우 리스트 형식보다 약 15초, 과제3의 경우 리스트형식보다 약 2분 40초 빠르게 수행함


In [1]:
import pandas as pd
import numpy as np 
from tqdm import tqdm
import FinanceDataReader as fdr

In [2]:
df_stock = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0] #데이터 불러오기
df_stock.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DL,210,기타 금융업,지주회사,1976-02-02,12월,전병욱,http://www.dlholdings.co.kr,서울특별시
1,DRB동일,4840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
2,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
3,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
4,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"최익훈, 정익희, 김회언 (각자 대표이사)",http://www.hdc-dvp.com,서울특별시


In [3]:
# 종목코드 바꾸기
df_stock['종목코드'] =df_stock['종목코드'].apply(lambda x : str(x).zfill(6))
lst_stock = df_stock.values.tolist()

# 과제1

In [5]:
result = {}

for row in tqdm(lst_stock):
    code,name, date = row[1],row[0], row[4]
    if date <= '2018-01-02':
        result[code]= name
        
OF = open('../data/assignment1','w')
for row in result:
    code, name = row[0],row[1]
    OF.write(f'{code}\t{name}\n')
OF.close()
        

100%|█████████████████████████████████████████████████████████████████████████| 2548/2548 [00:00<00:00, 1296661.80it/s]


# 과제2

In [15]:
dic_code2date = {}

for code in tqdm(result.keys()):
    stock = fdr.DataReader(code,start = '2018-01-01', end = '2021-12-31')

    lst_stock = stock.values.tolist()
    lst_date = stock.index
    
    for date, row in zip(lst_date, lst_stock):
        close, volume, trading = row[3], row[4], row[3]*row[4]
        
        if trading >= 1000000000000:
            if code not in dic_code2date.keys():
                dic_code2date[code] = []
                dic_code2date[code].append(date)
            else:
                dic_code2date[code].append(date)
                
OF = open('../data/assignment2.txt', 'w')
for code,date in dic_code2date.items():
    OF.write(f'{code}\t{date}\n')
OF.close()

100%|██████████████████████████████████████████████████████████████████████████████| 1968/1968 [03:09<00:00, 10.38it/s]


# 과제3

In [16]:
dic_code2date

{'096770': [Timestamp('2020-08-07 00:00:00'),
  Timestamp('2021-01-06 00:00:00'),
  Timestamp('2021-01-11 00:00:00'),
  Timestamp('2021-02-01 00:00:00'),
  Timestamp('2021-02-15 00:00:00'),
  Timestamp('2021-04-12 00:00:00'),
  Timestamp('2021-07-01 00:00:00')],
 '030790': [Timestamp('2018-07-11 00:00:00'),
  Timestamp('2019-04-05 00:00:00'),
  Timestamp('2019-04-08 00:00:00'),
  Timestamp('2019-04-09 00:00:00'),
  Timestamp('2019-04-22 00:00:00'),
  Timestamp('2020-01-17 00:00:00')],
 '005930': [Timestamp('2018-05-04 00:00:00'),
  Timestamp('2018-05-08 00:00:00'),
  Timestamp('2018-05-30 00:00:00'),
  Timestamp('2018-05-31 00:00:00'),
  Timestamp('2018-06-14 00:00:00'),
  Timestamp('2018-12-13 00:00:00'),
  Timestamp('2019-01-25 00:00:00'),
  Timestamp('2019-02-28 00:00:00'),
  Timestamp('2019-05-28 00:00:00'),
  Timestamp('2019-10-24 00:00:00'),
  Timestamp('2019-11-26 00:00:00'),
  Timestamp('2019-12-12 00:00:00'),
  Timestamp('2019-12-17 00:00:00'),
  Timestamp('2020-01-08 00:00:00

In [18]:
dic_result = {}
OF = open('../data/assignment3.txt','w')

for row in tqdm(dic_code2date.keys()):
    code = row
    stock = fdr.DataReader(code,start = '2018-01-01', end = '2021-12-31')
    stock.index = stock.index.strftime('%Y%m%d')

    lst_stock = stock.values.tolist()
    lst_date = stock.index
    
    for i,row in enumerate(lst_stock):
        date = row[0]
        # 10일간의 데이터를 모으지 못하는 경우
        if i < 9 :
            continue
            
        if date not in lst_date:
            continue
            
        sub_stock = stock[i-9:i+1]
        change = stock[i+1][-1]
        
        lst_result = []
        for row in sub_stock:
            open, high, low, close, volume = row[1:6]
            trading = row[4] * row[5]
            lst_result += [open,high,low,close,volume,trading]
        result = ','.join(map(str,lst_result))
        ratio = str(int(lst_stock[i+1][-1] > 0.02))
        
        
        OF.write('\t'.join(map(str,[code,date,result,ratio]))) + '\n'
OF.close()
        
    

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:09<00:00,  8.59it/s]
